In [4]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, accuracy_score,f1_score,recall_score,precision_score, confusion_matrix, mean_squared_error,roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
import xgboost as xgb



ModuleNotFoundError: No module named 'xgboost'

In [6]:
# !pip install xgboost

In [ ]:
def data_cleaning_demo(df):
    df=df.dropna(subset=['CustomerID'])
    # DEMOGRAPHICS 
    if('InsuredGender' in df.columns):
        df['InsuredGender']=np.where(df['InsuredGender']=='NA',np.nan,df['InsuredGender'])
        
    if('InsuredZipCode' in df.columns):
        df=df.drop(['InsuredZipCode'],axis=1)
        
    # POLICY INFO
    if('InsurancePolicyNumber' in df.columns):
        df=df.drop(['InsurancePolicyNumber'],axis=1)
        
    if('PolicyAnnualPremium' in df.columns):    
        df['PolicyAnnualPremium']=np.where(df['PolicyAnnualPremium']==-1,
                                            np.nan,df['PolicyAnnualPremium'])
    if('DateOfPolicyCoverage' in df.columns):
        df['DateOfPolicyCoverage']=pd.to_datetime(df['DateOfPolicyCoverage'],format='%Y-%m-%d')
        
    # CLAIM
    if('TypeOfCollission' in df.columns):
        df['TypeOfCollission']=np.where(df['TypeOfCollission']=='?',
                                    np.nan,df['TypeOfCollission'])
    if('IncidentTime' in df.columns):
        df['IncidentTime']=np.where(df['IncidentTime']==-5,
                                np.nan,df['IncidentTime'])
    if('PropertyDamage' in df.columns):
        df['PropertyDamage']=np.where(df['PropertyDamage']=='?',
                                  np.nan,df['PropertyDamage'])
    if('PoliceReport' in df.columns):
        df['PoliceReport']=np.where(df['PoliceReport']=='?',
                                np.nan,df['PoliceReport'])
    if('AmountOfTotalClaim' in df.columns):
        df['AmountOfTotalClaim']=np.where(df['AmountOfTotalClaim']=='MISSEDDATA',
                                              np.nan,df['AmountOfTotalClaim'])
        df['AmountOfTotalClaim']=df['AmountOfTotalClaim'].astype(float)
        
    if('DateOfIncident' in df.columns):   
        df['DateOfIncident']=pd.to_datetime(df['DateOfIncident'],format='%Y-%m-%d')
        
    if('IncidentAddress' in df.columns):
        df.drop(['IncidentAddress'],axis=1,inplace=True)
    
    # VEHICLE
    if('VehicleAttributeDetails' in df.columns):
        df['VehicleAttributeDetails']=np.where(df['VehicleAttributeDetails']=='???',
                                              np.nan,df['VehicleAttributeDetails'])

    return df

In [ ]:
def post_processing(df_train,df_test):
    
        
    df_train_obj=df_train.select_dtypes(include=['object'])
    df_train_num=df_train.select_dtypes(exclude=['object','datetime'])
    df_train_datetime=df_train.select_dtypes(include=['datetime'])
    num_cols=df_train_num.columns.tolist()
    cat_cols=df_train_obj.columns.tolist()
    datetime_cols=df_train_datetime.columns.tolist()
    print("object columns ::",cat_cols)
    print("numeric columns ::",num_cols)
    print("datetime columns ::",datetime_cols)

        
    imp_mean = SimpleImputer(strategy='mean')
    imp_mode = SimpleImputer(strategy='most_frequent')
    if(len(num_cols))>0:
        df_train_num=pd.DataFrame(imp_mean.fit_transform(df_train_num),columns=df_train_num.columns)
    if(len(cat_cols))>0:
        df_train_obj=pd.DataFrame(imp_mode.fit_transform(df_train_obj),columns=df_train_obj.columns)
        
    df_train_pp=pd.concat([df_train_obj,df_train_num,df_train_datetime],axis=1)
    
    
    oh_enc = OneHotEncoder(handle_unknown='ignore')
    cat_cols_onc=[i for i in cat_cols if i not in ('CustomerID','InsuredZipCode')]
    if(len(cat_cols_onc)>0):
        print("cat_cols_onc. ::",cat_cols_onc)
        df_train_enc=pd.DataFrame(oh_enc.fit_transform(df_train_pp[cat_cols_onc]).toarray(),
                                  columns=oh_enc.get_feature_names_out())
        df_train_pp=df_train_pp.drop(cat_cols_onc,axis=1)
        df_train_pp=pd.concat([df_train_pp,df_train_enc],axis=1)
    
    
    
    
    df_test_num=df_test[num_cols]
    df_test_obj=df_test[cat_cols]
    df_test_datetime=df_test[datetime_cols]
    
    if(len(num_cols))>0:
        df_test_num=pd.DataFrame(imp_mean.transform(df_test_num),columns=df_test_num.columns)
    if(len(cat_cols))>0:
        df_test_obj=pd.DataFrame(imp_mode.transform(df_test_obj),columns=df_test_obj.columns)
        
    df_test_pp=pd.concat([df_test_obj,df_test_num,df_test_datetime],axis=1)
    
    if(len(cat_cols_onc)>0):
        df_test_enc=pd.DataFrame(oh_enc.transform(df_test_pp[cat_cols_onc]).toarray(),
                                  columns=oh_enc.get_feature_names_out())
        df_test_pp=df_test_pp.drop(cat_cols_onc,axis=1)
        df_test_pp=pd.concat([df_test_pp,df_test_enc],axis=1)
    
    
    
    return df_train_pp,df_test_pp
    
    

# demographics

In [ ]:
df_train_demo=pd.read_csv('../TrainData/Train_Demographics.csv')
print(df_train_demo.shape)
df_train_demo.head()

In [ ]:
df_train_demo=data_cleaning_demo(df_train_demo)
print(df_train_demo.shape)
df_train_demo.head()

In [ ]:
df_test_demo=pd.read_csv('../TestData/Test_Demographics.csv')
print(df_test_demo.shape)
df_test_demo.head()

In [ ]:
df_test_demo=data_cleaning_demo(df_test_demo)
print(df_test_demo.shape)
df_test_demo.head()

In [ ]:
df_train_demo,df_test_demo=post_processing(df_train_demo,df_test_demo)

In [ ]:
df_train_demo.head()

In [ ]:
df_test_demo.head()

# Policy info

In [17]:
df_train_pi=pd.read_csv('../TrainData/Train_Policy.csv')
print(df_train_pi.shape)
df_train_pi.head()

(28836, 10)


,InsurancePolicyNumber,CustomerLoyaltyPeriod,DateOfPolicyCoverage,InsurancePolicyState,Policy_CombinedSingleLimit,Policy_Deductible,PolicyAnnualPremium,UmbrellaLimit,InsuredRelationship,CustomerID
0,110122,328,2014-10-17,State3,250/500,1000,1406.91,0,husband,Cust1001
1,110125,256,1990-05-25,State1,250/500,2000,1415.74,6000000,unmarried,Cust1004
2,110126,228,2014-06-06,State1,500/1000,1000,1583.91,6000000,unmarried,Cust1005
3,110127,256,2006-10-12,State3,250/500,1000,1351.10,0,unmarried,Cust1006
4,110128,137,2000-06-04,State2,250/500,1000,1333.35,0,husband,Cust1007


In [32]:
df_train_pi=data_cleaning_demo(df_train_pi)
print(df_train_pi.shape)
df_train_pi.head()

(28836, 9)


,CustomerLoyaltyPeriod,DateOfPolicyCoverage,InsurancePolicyState,Policy_CombinedSingleLimit,Policy_Deductible,PolicyAnnualPremium,UmbrellaLimit,InsuredRelationship,CustomerID
0,328,2014-10-17,State3,250/500,1000,1406.91,0,husband,Cust1001
1,256,1990-05-25,State1,250/500,2000,1415.74,6000000,unmarried,Cust1004
2,228,2014-06-06,State1,500/1000,1000,1583.91,6000000,unmarried,Cust1005
3,256,2006-10-12,State3,250/500,1000,1351.10,0,unmarried,Cust1006
4,137,2000-06-04,State2,250/500,1000,1333.35,0,husband,Cust1007


In [33]:
df_test_pi=pd.read_csv('../TestData/Test_Policy.csv')
print(df_test_pi.shape)
df_test_pi.head()

(8912, 10)


,InsurancePolicyNumber,CustomerLoyaltyPeriod,DateOfPolicyCoverage,InsurancePolicyState,Policy_CombinedSingleLimit,Policy_Deductible,PolicyAnnualPremium,UmbrellaLimit,InsuredRelationship,CustomerID
0,110124,134,2000-09-06,State3,100/300,2000,1413.14,5000000,own-child,Cust1003
1,110129,165,1990-02-03,State1,100/300,1000,1137.03,0,unmarried,Cust1008
2,110137,473,1992-10-19,State2,100/300,2000,1131.40,0,other-relative,Cust1016
3,110140,160,2014-12-28,State3,500/1000,500,1374.22,0,other-relative,Cust1019
4,110141,196,1992-08-02,State2,500/1000,2000,1475.73,0,own-child,Cust1020


In [34]:
df_test_pi=data_cleaning_demo(df_test_pi)
print(df_test_pi.shape)
df_test_pi.head()

(8912, 9)


,CustomerLoyaltyPeriod,DateOfPolicyCoverage,InsurancePolicyState,Policy_CombinedSingleLimit,Policy_Deductible,PolicyAnnualPremium,UmbrellaLimit,InsuredRelationship,CustomerID
0,134,2000-09-06,State3,100/300,2000,1413.14,5000000,own-child,Cust1003
1,165,1990-02-03,State1,100/300,1000,1137.03,0,unmarried,Cust1008
2,473,1992-10-19,State2,100/300,2000,1131.40,0,other-relative,Cust1016
3,160,2014-12-28,State3,500/1000,500,1374.22,0,other-relative,Cust1019
4,196,1992-08-02,State2,500/1000,2000,1475.73,0,own-child,Cust1020


In [35]:
df_train_pi,df_test_pi=post_processing(df_train_pi,df_test_pi)

object columns :: ['InsurancePolicyState', 'Policy_CombinedSingleLimit', 'InsuredRelationship', 'CustomerID']
numeric columns :: ['CustomerLoyaltyPeriod', 'Policy_Deductible', 'PolicyAnnualPremium', 'UmbrellaLimit']
datetime columns :: ['DateOfPolicyCoverage']
cat_cols_onc. :: ['InsurancePolicyState', 'Policy_CombinedSingleLimit', 'InsuredRelationship']


In [36]:
print(df_test_pi.shape)
df_test_pi.head()

(8912, 24)


,CustomerID,CustomerLoyaltyPeriod,Policy_Deductible,PolicyAnnualPremium,UmbrellaLimit,DateOfPolicyCoverage,InsurancePolicyState_State1,InsurancePolicyState_State2,InsurancePolicyState_State3,Policy_CombinedSingleLimit_100/1000,...,Policy_CombinedSingleLimit_250/500,Policy_CombinedSingleLimit_500/1000,Policy_CombinedSingleLimit_500/300,Policy_CombinedSingleLimit_500/500,InsuredRelationship_husband,InsuredRelationship_not-in-family,InsuredRelationship_other-relative,InsuredRelationship_own-child,InsuredRelationship_unmarried,InsuredRelationship_wife
0,Cust1003,134.0,2000.0,1413.14,5000000.0,2000-09-06,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Cust1008,165.0,1000.0,1137.03,0.0,1990-02-03,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Cust1016,473.0,2000.0,1131.40,0.0,1992-10-19,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,Cust1019,160.0,500.0,1374.22,0.0,2014-12-28,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,Cust1020,196.0,2000.0,1475.73,0.0,1992-08-02,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# Claim

In [37]:
df_train_claim=pd.read_csv('../TrainData/Train_Claim.csv')
print(df_train_claim.shape)
df_train_claim.head()

(28836, 19)


,CustomerID,DateOfIncident,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,IncidentTime,NumberOfVehicles,PropertyDamage,BodilyInjuries,Witnesses,PoliceReport,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage
0,Cust10000,2015-02-03,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City1,Location 1311,17,3,?,1,0,?,65501,13417,6071,46013
1,Cust10001,2015-02-02,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City5,Location 1311,10,3,YES,2,1,YES,61382,15560,5919,39903
2,Cust10002,2015-01-15,Single Vehicle Collision,Side Collision,Minor Damage,Other,State8,City6,Location 2081,22,1,YES,2,3,NO,66755,11630,11630,43495
3,Cust10003,2015-01-19,Single Vehicle Collision,Side Collision,Minor Damage,Other,State9,City6,Location 2081,22,1,YES,2,3,NO,66243,12003,12003,42237
4,Cust10004,2015-01-09,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,State8,City6,Location 1695,10,1,NO,2,1,YES,53544,8829,7234,37481


In [41]:
df_train_claim=data_cleaning_demo(df_train_claim)
print(df_train_claim.shape)
df_train_claim.head()

(28836, 18)


,CustomerID,DateOfIncident,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentTime,NumberOfVehicles,PropertyDamage,BodilyInjuries,Witnesses,PoliceReport,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage
0,Cust10000,2015-02-03,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City1,17.0,3,NaN,1,0,NaN,65501.0,13417,6071,46013
1,Cust10001,2015-02-02,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City5,10.0,3,YES,2,1,YES,61382.0,15560,5919,39903
2,Cust10002,2015-01-15,Single Vehicle Collision,Side Collision,Minor Damage,Other,State8,City6,22.0,1,YES,2,3,NO,66755.0,11630,11630,43495
3,Cust10003,2015-01-19,Single Vehicle Collision,Side Collision,Minor Damage,Other,State9,City6,22.0,1,YES,2,3,NO,66243.0,12003,12003,42237
4,Cust10004,2015-01-09,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,State8,City6,10.0,1,NO,2,1,YES,53544.0,8829,7234,37481


In [38]:
df_test_claim=pd.read_csv('../TestData/Test_Claim.csv')
print(df_test_claim.shape)
df_test_claim.head()

(8912, 19)


,CustomerID,DateOfIncident,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,IncidentTime,NumberOfVehicles,PropertyDamage,BodilyInjuries,Witnesses,PoliceReport,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage
0,Cust10008,2015-02-05,Multi-vehicle Collision,Front Collision,Minor Damage,Ambulance,State5,City2,Location 1354,4,3,NO,0,0,?,68354,6835,8059,53460
1,Cust10010,2015-01-13,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,State9,City5,Location 1383,16,1,?,1,1,?,55270,8113,5240,41917
2,Cust10015,2015-01-05,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,State5,City2,Location 2030,20,3,NO,0,1,?,59515,7490,9110,42915
3,Cust10020,2015-01-03,Vehicle Theft,?,Trivial Damage,None,State7,City5,Location 1449,10,1,?,2,1,?,4941,494,866,3581
4,Cust1003,2015-02-22,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,State5,City2,Location 1916,7,3,NO,2,3,NO,34650,7700,3850,23100


In [42]:
df_test_claim=data_cleaning_demo(df_test_claim)
print(df_test_claim.shape)
df_test_claim.head()

(8912, 18)


,CustomerID,DateOfIncident,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentTime,NumberOfVehicles,PropertyDamage,BodilyInjuries,Witnesses,PoliceReport,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage
0,Cust10008,2015-02-05,Multi-vehicle Collision,Front Collision,Minor Damage,Ambulance,State5,City2,4.0,3,NO,0,0,NaN,68354.0,6835,8059,53460
1,Cust10010,2015-01-13,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,State9,City5,16.0,1,NaN,1,1,NaN,55270.0,8113,5240,41917
2,Cust10015,2015-01-05,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,State5,City2,20.0,3,NO,0,1,NaN,59515.0,7490,9110,42915
3,Cust10020,2015-01-03,Vehicle Theft,NaN,Trivial Damage,None,State7,City5,10.0,1,NaN,2,1,NaN,4941.0,494,866,3581
4,Cust1003,2015-02-22,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,State5,City2,7.0,3,NO,2,3,NO,34650.0,7700,3850,23100


In [44]:
df_train_claim,df_test_claim=post_processing(df_train_claim,df_test_claim)

object columns :: ['CustomerID', 'TypeOfIncident', 'TypeOfCollission', 'SeverityOfIncident', 'AuthoritiesContacted', 'IncidentState', 'IncidentCity', 'PropertyDamage', 'Witnesses', 'PoliceReport']
numeric columns :: ['IncidentTime', 'NumberOfVehicles', 'BodilyInjuries', 'AmountOfTotalClaim', 'AmountOfInjuryClaim', 'AmountOfPropertyClaim', 'AmountOfVehicleDamage']
datetime columns :: ['DateOfIncident']
cat_cols_onc. :: ['TypeOfIncident', 'TypeOfCollission', 'SeverityOfIncident', 'AuthoritiesContacted', 'IncidentState', 'IncidentCity', 'PropertyDamage', 'Witnesses', 'PoliceReport']


In [45]:
print(df_train_claim.shape)
df_train_claim.head()

(28836, 48)


,CustomerID,IncidentTime,NumberOfVehicles,BodilyInjuries,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage,DateOfIncident,TypeOfIncident_Multi-vehicle Collision,...,IncidentCity_City7,PropertyDamage_NO,PropertyDamage_YES,Witnesses_0,Witnesses_1,Witnesses_2,Witnesses_3,Witnesses_MISSINGVALUE,PoliceReport_NO,PoliceReport_YES
0,Cust10000,17.0,3.0,1.0,65501.0,13417.0,6071.0,46013.0,2015-02-03,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Cust10001,10.0,3.0,2.0,61382.0,15560.0,5919.0,39903.0,2015-02-02,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,Cust10002,22.0,1.0,2.0,66755.0,11630.0,11630.0,43495.0,2015-01-15,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,Cust10003,22.0,1.0,2.0,66243.0,12003.0,12003.0,42237.0,2015-01-19,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,Cust10004,10.0,1.0,2.0,53544.0,8829.0,7234.0,37481.0,2015-01-09,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


# vehicle

## vehicle train

In [53]:
df_train_veh=pd.read_csv('../TrainData/Train_Vehicle.csv')
print(df_train_veh.shape)
df_train_veh.head()

(115344, 3)


,CustomerID,VehicleAttribute,VehicleAttributeDetails
0,Cust20179,VehicleID,Vehicle8898
1,Cust21384,VehicleModel,Malibu
2,Cust33335,VehicleMake,Toyota
3,Cust27118,VehicleModel,Neon
4,Cust13038,VehicleID,Vehicle30212


In [54]:
df_train_veh=data_cleaning_demo(df_train_veh)
print(df_train_veh.shape)
df_train_veh.head()

(115344, 3)


,CustomerID,VehicleAttribute,VehicleAttributeDetails
0,Cust20179,VehicleID,Vehicle8898
1,Cust21384,VehicleModel,Malibu
2,Cust33335,VehicleMake,Toyota
3,Cust27118,VehicleModel,Neon
4,Cust13038,VehicleID,Vehicle30212


In [55]:
df_train_veh=df_train_veh.pivot(index=['CustomerID'],columns=['VehicleAttribute'],values=['VehicleAttributeDetails'])
df_train_veh.columns=['_'.join(x).strip() for x in df_train_veh.columns]
df_train_veh=df_train_veh.reset_index()
df_train_veh=df_train_veh.drop(['VehicleAttributeDetails_VehicleID'],axis=1)
df_train_veh['VehicleAttributeDetails_VehicleYOM']=pd.to_datetime(df_train_veh['VehicleAttributeDetails_VehicleYOM'
                                                                              ],format='%Y')
print(df_train_veh.shape)
df_train_veh.head()

(28836, 4)


,CustomerID,VehicleAttributeDetails_VehicleMake,VehicleAttributeDetails_VehicleModel,VehicleAttributeDetails_VehicleYOM
0,Cust10000,Audi,A5,2008-01-01
1,Cust10001,Audi,A5,2006-01-01
2,Cust10002,Volkswagen,Jetta,1999-01-01
3,Cust10003,Volkswagen,Jetta,2003-01-01
4,Cust10004,Toyota,CRV,2010-01-01


## vehicle test

In [56]:
df_test_veh=pd.read_csv('../TestData/Test_Vehicle.csv')
print(df_test_veh.shape)
df_test_veh.head()

(35648, 3)


,CustomerID,VehicleAttribute,VehicleAttributeDetails
0,Cust28465,VehicleModel,92x
1,Cust20319,VehicleModel,Jetta
2,Cust3683,VehicleID,Vehicle22709
3,Cust29385,VehicleID,Vehicle20347
4,Cust34403,VehicleModel,Camry


In [57]:
df_test_veh=data_cleaning_demo(df_test_veh)
print(df_test_veh.shape)
df_test_veh.head()

(35648, 3)


,CustomerID,VehicleAttribute,VehicleAttributeDetails
0,Cust28465,VehicleModel,92x
1,Cust20319,VehicleModel,Jetta
2,Cust3683,VehicleID,Vehicle22709
3,Cust29385,VehicleID,Vehicle20347
4,Cust34403,VehicleModel,Camry


In [58]:
df_test_veh=df_test_veh.pivot(index=['CustomerID'],columns=['VehicleAttribute'],values=['VehicleAttributeDetails'])
df_test_veh.columns=['_'.join(x).strip() for x in df_test_veh.columns]
df_test_veh=df_test_veh.reset_index()
df_test_veh=df_test_veh.drop(['VehicleAttributeDetails_VehicleID'],axis=1)
df_test_veh['VehicleAttributeDetails_VehicleYOM']=pd.to_datetime(df_test_veh['VehicleAttributeDetails_VehicleYOM'
                                                                              ],format='%Y')
print(df_test_veh.shape)
df_test_veh.head()

(8912, 4)


,CustomerID,VehicleAttributeDetails_VehicleMake,VehicleAttributeDetails_VehicleModel,VehicleAttributeDetails_VehicleYOM
0,Cust10008,Volkswagen,Passat,1995-01-01
1,Cust10010,Nissan,Ultima,2006-01-01
2,Cust10015,Suburu,Impreza,2010-01-01
3,Cust10020,Accura,TL,2009-01-01
4,Cust1003,Dodge,RAM,2007-01-01


In [59]:
df_train_veh,df_test_veh=pp_vehicle_info(df_train_veh,df_test_veh)

object columns :: ['CustomerID', 'VehicleAttributeDetails_VehicleMake', 'VehicleAttributeDetails_VehicleModel']
numeric columns :: []
datetime columns :: ['VehicleAttributeDetails_VehicleYOM']
cat_cols_onc. :: ['VehicleAttributeDetails_VehicleMake', 'VehicleAttributeDetails_VehicleModel']


In [61]:
print(df_train_veh.shape)
df_train_veh.head()

(28836, 55)


,CustomerID,VehicleAttributeDetails_VehicleYOM,VehicleAttributeDetails_VehicleMake_Accura,VehicleAttributeDetails_VehicleMake_Audi,VehicleAttributeDetails_VehicleMake_BMW,VehicleAttributeDetails_VehicleMake_Chevrolet,VehicleAttributeDetails_VehicleMake_Dodge,VehicleAttributeDetails_VehicleMake_Ford,VehicleAttributeDetails_VehicleMake_Honda,VehicleAttributeDetails_VehicleMake_Jeep,...,VehicleAttributeDetails_VehicleModel_Pathfinder,VehicleAttributeDetails_VehicleModel_RAM,VehicleAttributeDetails_VehicleModel_RSX,VehicleAttributeDetails_VehicleModel_Silverado,VehicleAttributeDetails_VehicleModel_TL,VehicleAttributeDetails_VehicleModel_Tahoe,VehicleAttributeDetails_VehicleModel_Ultima,VehicleAttributeDetails_VehicleModel_Wrangler,VehicleAttributeDetails_VehicleModel_X5,VehicleAttributeDetails_VehicleModel_X6
0,Cust10000,2008-01-01,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Cust10001,2006-01-01,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Cust10002,1999-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Cust10003,2003-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Cust10004,2010-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# target

In [66]:
train_target=pd.read_csv('../TrainData/Traindata_with_Target.csv')
train_target=train_target.dropna(subset=['CustomerID'])
print(train_target.shape)
train_target.head()

(28836, 2)


,CustomerID,ReportedFraud
0,Cust20065,N
1,Cust37589,N
2,Cust24312,N
3,Cust5493,Y
4,Cust7704,Y


In [67]:
le=LabelEncoder()
train_target['ReportedFraud']=le.fit_transform(train_target['ReportedFraud'])
train_target.head()

,CustomerID,ReportedFraud
0,Cust20065,0
1,Cust37589,0
2,Cust24312,0
3,Cust5493,1
4,Cust7704,1


In [68]:
print(list(le.classes_))
print(le.transform(list(le.classes_)))
print(le.inverse_transform(le.transform(list(le.classes_))))

['N', 'Y']
[0 1]
['N' 'Y']


In [69]:
test_target=pd.read_csv('../TestData/Test.csv')
test_target=test_target.dropna(subset=['CustomerID'])
print(test_target.shape)
test_target.head()

(8912, 1)


,CustomerID
0,Cust10008
1,Cust10010
2,Cust10015
3,Cust10020
4,Cust1003


# MERGING ALL DATA SETS

In [74]:
train_target.shape

(28836, 2)

In [84]:
test_target.shape

(8912, 1)

In [159]:
df_train_merged=train_target.merge(df_train_demo,how='inner',on=['CustomerID'],indicator='in1') \
                            .merge(df_train_pi,how='inner',on=['CustomerID'],indicator='in2') \
                            .merge(df_train_claim,how='inner',on=['CustomerID'],indicator='in3') \
                            .merge(df_train_veh,how='inner',on=['CustomerID'],indicator='in4')

# print(df_train_merged['in1'].value_counts())
# print(df_train_merged['in2'].value_counts())
# print(df_train_merged['in3'].value_counts())
# print(df_train_merged['in4'].value_counts())
df_train_merged.drop(['CustomerID','in1','in2','in3','in4'],axis=1,inplace=True)
print(df_train_merged.shape)
df_train_merged['no_days_incident_vehicleYOM']=(df_train_merged['DateOfIncident'] \
                                                -df_train_merged['VehicleAttributeDetails_VehicleYOM']).dt.days
df_train_merged['no_days_incident_PolicyCoverage']=(df_train_merged['DateOfIncident'] \
                                                -df_train_merged['DateOfPolicyCoverage']).dt.days
df_train_merged=df_train_merged.drop(['DateOfIncident',
                      'VehicleAttributeDetails_VehicleYOM','DateOfPolicyCoverage'],axis=1)
df_train_merged=df_train_merged.query("no_days_incident_vehicleYOM>=0 and no_days_incident_PolicyCoverage>=0")
print(df_train_merged.shape)
df_train_merged.head()

(28836, 172)
(28834, 171)


,ReportedFraud,InsuredAge,CapitalGains,CapitalLoss,InsuredGender_FEMALE,InsuredGender_MALE,InsuredEducationLevel_Associate,InsuredEducationLevel_College,InsuredEducationLevel_High School,InsuredEducationLevel_JD,...,VehicleAttributeDetails_VehicleModel_RSX,VehicleAttributeDetails_VehicleModel_Silverado,VehicleAttributeDetails_VehicleModel_TL,VehicleAttributeDetails_VehicleModel_Tahoe,VehicleAttributeDetails_VehicleModel_Ultima,VehicleAttributeDetails_VehicleModel_Wrangler,VehicleAttributeDetails_VehicleModel_X5,VehicleAttributeDetails_VehicleModel_X6,no_days_incident_vehicleYOM,no_days_incident_PolicyCoverage
0,0,58.0,0.0,-42700.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3660,5900
1,0,34.0,51300.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1876,5298
2,0,52.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4761,5318
3,1,25.0,47400.0,-56100.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1467,8393
4,1,27.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1882,4904


In [147]:
# df_train_merged.query("CustomerID=='Cust1579'")[['DateOfIncident',
#                       'VehicleAttributeDetails_VehicleYOM','DateOfPolicyCoverage','ReportedFraud']]
# Cust35251
# Cust1579

In [160]:
df_test_merged=test_target.merge(df_test_demo,how='inner',on=['CustomerID'],indicator='in1') \
                            .merge(df_test_pi,how='inner',on=['CustomerID'],indicator='in2') \
                            .merge(df_test_claim,how='inner',on=['CustomerID'],indicator='in3') \
                            .merge(df_test_veh,how='inner',on=['CustomerID'],indicator='in4')

df_test_merged.drop(['CustomerID','in1','in2','in3','in4'],axis=1,inplace=True)
df_test_merged['no_days_incident_vehicleYOM']=(df_test_merged['DateOfIncident'] \
                                                -df_test_merged['VehicleAttributeDetails_VehicleYOM']).dt.days
df_test_merged['no_days_incident_PolicyCoverage']=(df_test_merged['DateOfIncident'] \
                                                -df_test_merged['DateOfPolicyCoverage']).dt.days
df_test_merged=df_test_merged.drop(['DateOfIncident',
                      'VehicleAttributeDetails_VehicleYOM','DateOfPolicyCoverage'],axis=1)
df_test_merged=df_test_merged.query("no_days_incident_vehicleYOM>=0 and no_days_incident_PolicyCoverage>=0")

print(df_test_merged.shape)
df_test_merged.head()

(8911, 170)


,InsuredAge,CapitalGains,CapitalLoss,InsuredGender_FEMALE,InsuredGender_MALE,InsuredEducationLevel_Associate,InsuredEducationLevel_College,InsuredEducationLevel_High School,InsuredEducationLevel_JD,InsuredEducationLevel_MD,...,VehicleAttributeDetails_VehicleModel_RSX,VehicleAttributeDetails_VehicleModel_Silverado,VehicleAttributeDetails_VehicleModel_TL,VehicleAttributeDetails_VehicleModel_Tahoe,VehicleAttributeDetails_VehicleModel_Ultima,VehicleAttributeDetails_VehicleModel_Wrangler,VehicleAttributeDetails_VehicleModel_X5,VehicleAttributeDetails_VehicleModel_X6,no_days_incident_vehicleYOM,no_days_incident_PolicyCoverage
0,27.0,56400.0,-57000.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7340,6115
1,40.0,56700.0,-65600.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3299,1160
2,39.0,30400.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1830,1633
3,38.0,68500.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2193,5228
4,29.0,35100.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2974,5282


In [161]:
X_train_all=df_train_merged.drop(['ReportedFraud'],axis=1)
Y_train_all=df_train_merged[['ReportedFraud']]
print(Y_train_all.value_counts(dropna=False,normalize=True)*100)

ReportedFraud
0                73.000624
1                26.999376
dtype: float64


In [162]:
X_train, X_test, y_train, y_test = train_test_split(X_train_all, Y_train_all, 
                                                    test_size=0.33, random_state=42,
                                                    stratify=Y_train_all)

In [163]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42, ...)

In [166]:
y_pred=xgb_model.predict(X_test)
pd.Series(y_pred).value_counts(dropna=False,normalize=True)*100

0    76.796974
1    23.203026
dtype: float64

In [167]:
print(confusion_matrix(y_test, y_pred))

[[6736  211]
 [ 572 1997]]


In [170]:
print('Precision: %.3f' % precision_score(y_test, y_pred))
print('Recall: %.3f' % recall_score(y_test, y_pred))
print('F1: %.3f' % f1_score(y_test, y_pred))
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

Precision: 0.904
Recall: 0.777
F1: 0.836
Accuracy: 0.918


In [1]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=[0,1],
        y=[0,1],
        name="TPR = FPR",
        line=dict(color="black", dash="dash")
    )
)
def PlotRocAuc(y_test, y_pred, color, model_name):
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    auc_score = roc_auc_score(y_test,y_pred)
    fig.add_trace(
        go.Scatter(
            x=fpr,
            y=tpr,
            name=f"{model_name}(AUC={auc_score})",
            marker=dict(color=color)
        )
    )
    
PlotRocAuc(y_test, y_pred, "green", "rf_clf")
fig.update_layout(title="ROC curve",
                  xaxis_title="False Positive Rate",
                  yaxis_title="True Positive Rate")

fig.show()

ModuleNotFoundError: No module named 'plotly'